In [81]:
import numpy as np
import re
import ast
import subprocess
from codes.utils.file import create_file_abs, create_parent_dir, dir_exists_abs, file_exists_absolute, join_dir, file_exists_current, file_remove_current
from codes.parameters import ASTRAL, NUMPY_EXTENSION, QUARTETS, SCRIPT_FOLDER, STATS_FOLDER, TRUE_FOLDER, WQFM, \
    COMPLETE, GENE_TREE, QUARTETS, INCOMPLETE, IMPUTED_LIST, OUTPUT_FOLDER, NUMPY_ARRAY, REMOVED_TAXA_RANGE, INPUT_FOLDER, \
    QUARTETS_EXTENSION, MODES_LIST, WORKING_FOLDER, METHODS, TERRACE_OUTPUT_FOLDER

In [82]:
file_path = "terrace_output/15-18.txt.raxml.consensusTreeMRE"
# !rm -r terrace_output/15-18.txt.raxml.consensusTreeMRE

In [83]:
# !./bin/raxml-ng --consense MRE --tree terrace_output/15-18.txt

In [84]:
file = open('terrace_output/15-18.txt.raxml.consensusTreeMRE')
lines = file.readlines()

In [85]:
line = lines[0]
line = re.sub(r'[0-9]', '', line)
line

'((PTE,MYO),((EQU,(((ECH,(PRO,LOX)),((CHO,DAS),(GAL,(ORN,(MAC,MON))))),(((MIC,OTO),(TAR,((CAL,NEW),(PON,(PAN,(GOR,HOM)))))),(TUP,((SPE,(CAV,(OCH,ORY))),(DIP,(MUS,RAT))))))),(ERI,(SOR,(CAN,FEL)))),(TUR,(SUS,(VIC,BOS))));\n'

In [86]:
file = open('terrace_output/15-18.txt.raxml.consensusTreeMRE', 'w')
file.writelines(line)
file.close()


In [87]:
true_tree = join_dir(TRUE_FOLDER, 'true-species.out.tree')
file_abs = join_dir('terrace_output/15-18.txt.raxml.consensusTreeMRE')

In [88]:
# get RF distance
result = ast.literal_eval(subprocess.check_output(['getFpFn.py', '-t', true_tree, '-e', file_abs], encoding='UTF-8'))
assert result[0] == result[1] == result[2]
dist = str(result[0])
print( dist)

0.35294117647058826
